In [9]:
# Clear up space before running
rm(list=ls())

In [10]:
# import libraries
library(bnlearn)
library(Rgraphviz)
library(stringr)

In [26]:
# This assumes BNLearn R is in a folder alongside and Inputs folder (containing the files to be run in bnLearn) 
#      and an Outputs folder (containg the bnLearn results)

# specialName is an identifer for all the files you want to run (if you want ot use all files in Inputs/, specialName=''),
#     this should not be the cnacer area name
# version is the named data version used  (in my case either 'v7' or 'v8')
# inputLst is the files you want to run bnLearn on
# doneLst is which of those files have already been acomplished
#      to rerun everything set doneLst = c()
# thres is the strength threshold needed for a connection to be inlcuded in the final output
# scoreMethod is the socring method to be used: should be either 'bge' or 'bde'
# graph tells if bnLearn should show a graph of the result as it gets made
# outDir is the directory in which the output csv should be saved in
specialName='Kobe Genes'
version='v7'
findFiles=paste('*',specialName,'*',version,'*',sep='')
inputLst = Sys.glob(file.path('Inputs',findFiles))
doneLst = Sys.glob(file.path('Outputs',findFiles))
thres=0.5
scoreMethod='bge'
graph=FALSE
outDir = 'Outputs/'

In [ ]:
inputLst

In [27]:
# runLst is a list of which files that are in the inputLst haven't already been done, and need to be run.
# just inputLst - doneLst
runLst = c()
for (elem in inputLst){
    str = strsplit(elem,'/')
    elem2 = str[[1]][2] 
    if(scoreMethod=='bde'){
        elem2 = paste('Outputs/',elem2,', MN.csv',sep='')
    }else{
        elem2 = paste('Outputs/',elem2,', G.csv',sep='')
    }
    elem2 = str_replace(elem2,'Pure ','')
    if(!elem2 %in% doneLst){
        runLst = c(runLst,elem)
    }
}

In [28]:
print(length(inputLst))
print(length(doneLst))
print(length(runLst))

[1] 1
[1] 1
[1] 1


In [ ]:
# Turn off warnings 
oldw <- getOption("warn")
options(warn = -1)

for (input in runLst){
    # read in input file
    data = read.csv(input,row.names=1,header=T,sep='\t')
    # discretize the data (only if using bde), 6 for heat, 3 for everything else
    if(scoreMethod=='bde'){
        LB<-c()
        i=1
        while(i <= ncol(data)){
            if(i==2){
               LB<-c(LB,6) 
            }
            else{
               LB<-c(LB,3) 
            }
            i=i+1
        }
        data = discretize(data, method = 'interval', breaks = LB)
    }
    
    # create a blackList for the data, making sure that Purity.Scores will allways be a root
    #     and that Heat.Scores will always be a node
    # all pairings within the blacklist are not allowd to happen
    bL<-c()
    lst = c(colnames(data))
    for(elem in lst){
        if(elem!='Purity.Scores' & elem!='Heat.Scores'){
            bL<-c(bL,elem,'Purity.Scores')
        }
    }
    for(elem in lst){
        if(elem!='Heat.Scores'){
            bL<-c(bL,'Heat.Scores',elem)
        }
    }
    # turn the blacklist into a matrix
    bL = matrix(bL, ncol = 2, byrow = TRUE)
    # run bnLearn, with 100 R, using 90% of the data, hc algorithm, the blacklist,and the chosen scoring method
    # to use the blacklist cpdag needs to be turned off (otherwise blacklist affects both sides of the pairing)
    boot = boot.strength(data = data, R = 100, m = trunc(.9 * nrow(data)), algorithm = 'hc',
                         algorithm.args = list(score = scoreMethod,blacklist=bL),cpdag = FALSE)
    
    # Save average of bootstraps for use in graphing (easier for me to create the graphs in python)
    findOldName=paste('.*',specialName,sep='')
    outName = sub(findOldName,specialName,input)
    if(scoreMethod=='bde'){
        outName = paste(outName,", MN",sep='')
    }
    else{
        outName = paste(outName,", G",sep='')
    }
    write.csv(boot,paste(outDir,outName,'.csv',sep=''))

    # If you want R to graph it
    if(graph){
        avg.boot = averaged.network(boot, threshold = thres)
        gr = graphviz.plot(avg.boot, shape = "ellipse",main=outName)
        renderGraph(gr)
    }
}
# turn warnings back on
options(warn = oldw)